## Data-driven Salary Analysis Using GlassDoor

Ricky Lim

In [1]:
import datetime

datetime.date.today().strftime("%B %d, %Y")

'December 21, 2017'

# Goal:

- What is the mean salary value for a researcher (onderzoeker) in The Netherlands ?
- What is the gap-ratio on the mean salary value between private and public sector on average in The Netherlands ?

# Methodology

- **Web-scraping** Glassdoor-specific query using `selenium` and `beautifulsoup (bs4)` python libraries
- **Extraction, Transformation, Loading (ETL)** salary data into DataFrame using `pandas` library
- **Visualization** using`bokeh`, and **descriptive statistics** on salary data using `pandas`

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

## Web-scraping

In [ ]:
from get_gd_salary import get_all_salaries

WEBSITE = "https://www.glassdoor.com/Salaries/" + \
          "netherlands-onderzoeker-salary-SRCH_IL.0,11_IN178_KO12,23_SDMC.htm"
    
onderzoeker_salaris = get_all_salaries(WEBSITE)

## Extraction, Transformation, Loading (ETL)

In [ ]:
import pandas as pd

salaris_pd = pd.DataFrame(onderzoeker_salaris, dtype="object")
salaris_pd.columns = ["job_title", "mean_salary"]
salaris_pd.head()
salaris_pd.describe()

In [ ]:
# parse company data from job_title
salaris_pd["title"], salaris_pd["bedrijf"] = salaris_pd["job_title"].\
    str.split(" bij ", 1).str

In [ ]:
salaris_pd.head()

In [ ]:
from re import sub

def annualize_salary(row):
    """
    unify salary values per year
    """
    # remove non-digit for dutch money (decimal point is "," instead of ".")
    money = sub(r"[^\d,]", "", row["mean_salary"])
    money = float(sub(r",", ".", money))
    title = row["title"]
    # convert monthly salary into annual with 12 times 
    if "mnd" in row["mean_salary"] or "maand" in title:
        money *= 12.0
        title = sub(r"- per maand", "", title)
    return title, money 

salaris_pd[["title", "mean_annual_salary"]] = salaris_pd.apply(annualize_salary, 
                                                               axis=1).apply(pd.Series)

# remove annual salary below minimum wages, assuming 1000 euros per maand
MIN_ANNUAL_SALARY = 12000
salaris_pd = salaris_pd[salaris_pd["mean_annual_salary"] >= MIN_ANNUAL_SALARY]

# aggregate salary values by company
company_pd = salaris_pd.groupby("bedrijf")["mean_annual_salary"].mean()
company_pd = pd.Series.to_frame(company_pd)
company_pd["company"] = company_pd.index
company_pd.head()

In [ ]:
# add label sector to a company, either public or private
import re

public = ["academisch", "academy", "university", "universiteit", 
          "hogeschool", "tu", "universitair", "umc", "ur", 
          "ministrie", "phd", "nwo", "rsm", "nations", 
          "gemeente", "rivm", "fontys", "fom", 
          "mc", "ministerie", "cwi",
          "society", "european", "agency", "netherlands"]

def ispublic(x):
    public_re = ""
    for _ in public:
        public_re += ".*%s.*|" % _
    public_re.rstrip("|")
    public_re = re.compile(public_re, re.IGNORECASE)
    if public_re.match(x).group():
        return "public" 
    return "private" 

company_pd["sector"] = company_pd.index.map(ispublic)
company_pd.head()

## Visualization, and descriptive statistics

In [ ]:
import math
from bokeh.models import ColumnDataSource

title="Mean Base Salary (Onderzoeker)" 
company_pd["color"] =["#abdda4" if s =="public" else 
                      "#2b83ba" for s in company_pd["sector"]]

source = ColumnDataSource(data=company_pd)

p = figure(x_range=list(company_pd["company"]), plot_width=1000, plot_height=800, 
           title=title)
p.vbar(x='company', top='mean_annual_salary', width=0.5, 
       source=source, legend="sector",
       line_color='white', fill_color="color")
p.xaxis.major_label_orientation = math.pi/2
p.yaxis.axis_label = "Mean Annual Salary in (€)"
p.xaxis.axis_label = "Company"
show(p)

company_pd.describe().round(2)

# Results

The mean of base salary for a researcher position in the Netherlands across private and public sectors is **€ 42,671.03**

## Public and private sector-specific 

In [ ]:
public_pd = company_pd[company_pd["sector"] == "public"]
private_pd = company_pd[company_pd["sector"] == "private"]

def salary_barplot(df, title):
    source = ColumnDataSource(data=df)
    p = figure(x_range=list(df["company"]), plot_width=1000, 
               plot_height=800, title=title)
    p.vbar(x='company', top='mean_annual_salary', width=0.8, 
           source=source, legend="sector",
           line_color='white', fill_color="color")
    p.xaxis.major_label_orientation = math.pi/2
    p.yaxis.axis_label = "Mean Annual Salary in (€)"
    p.xaxis.axis_label = "Company"
    show(p)

salary_barplot(public_pd, "Public Sector Mean Base Salary")
public_pd.describe().round(2)


In [ ]:
salary_barplot(private_pd, "Private Sector Mean Base Salary")
private_pd.describe().round(2)

## Difference between private and public sector

In [ ]:
import numpy as np
np.round(private_pd["mean_annual_salary"].mean()/
         public_pd["mean_annual_salary"].mean(), 2)

The mean base salary value on average in *private* sector is **1.3x** higher than in *public* sector in the Netherlands, given this dataset

# Acknowledgement

- glassdoor: https://www.glassdoor.nl
- selenium: http://selenium-python.readthedocs.io/
- beautifulsoup: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
- pandas: https://pandas.pydata.org/